In [1]:
from Distributed_Agent import Functions
from Distributed_Agent import Analysis

from openpyxl import Workbook
import importlib

from numba import njit

In [3]:
N_density = 10/(45**2)
N_density

0.0049382716049382715

In [16]:
import numpy as np

rho = 7.65
N_density = 10/(45**2)

rho = np.clip(rho, N_density, 8)
# rho = min(rho, 8)                       ### should debug in training
# rho = max(rho, N_density)
rho

7.65

In [ ]:
N = 10                                 # Number of agents
L = 45                                  # The length of the simulation box
Alphas = [-0.5, +0.3, 1.0, -1000]       # Hamiltonian constant coefficients
learning_rate = 1e-5                    # Learning rate in model training
discount_rate = 0.98                    # The effect of future decisions on current decisions
batch_size    = 20                      # Selected package of replay_memory to perform training
steps_per_train = 10                    # The training function is called once every ... steps
Parameters = [N, L, Alphas, learning_rate, discount_rate, batch_size, steps_per_train]

requesting = True                       # Enable the sending request function
moving     = True                       # Enable the move function to agents
training   = True                       # Enable each agent model training function
Func_params  = [requesting, moving, training]

buildings_type = "random"               # Type of placement of buildings (random or regular)
num_buildings = 00                       # Number of buildings in environment
num_streets   = 0                        # Number of streets in environment
ENV_Parameters = [L, buildings_type, num_buildings, num_streets]

Initialize_model_path = f'./Saved Model/Different Model Training/models with alpha4 = {Alphas[3]}/L={L}/model_H_best_weight.keras'


for seed in range(30, 31):
    ParticleExcel = Workbook(); del ParticleExcel['Sheet']
    AveragedExcel = Workbook(); del AveragedExcel['Sheet']


    plot_env, Agents = Functions.Initializer(ENV_Parameters, Parameters, Func_params, Initialize_model_path, seed)
    AveragedResults, ParticleResults = Functions.base_model(plot_env, Agents, MAX_step=10)
    Functions.SaveExcel(ParticleExcel, AveragedExcel, 'Base', AveragedResults, ParticleResults)
    print("  Finish Base...")

    Func_params  = [False, moving, training]
    plot_env, Agents = Functions.Initializer(ENV_Parameters, Parameters, Func_params, Initialize_model_path, seed)
    AveragedResults, ParticleResults = Functions.AI_model(plot_env, Agents, MAX_step=10)
    Functions.SaveExcel(ParticleExcel, AveragedExcel, 'Just Model', AveragedResults, ParticleResults)
    print("  Finish Just Model...")

    Func_params = [True, moving, training]
    plot_env, Agents = Functions.Initializer(ENV_Parameters, Parameters, Func_params, Initialize_model_path, seed)
    AveragedResults, ParticleResults = Functions.AI_model(plot_env, Agents, MAX_step=10)
    Functions.SaveExcel(ParticleExcel, AveragedExcel, 'Send Request', AveragedResults, ParticleResults)
    print("  Finish Send Request!")
    

    path = f"./All Results/Datas/data {seed}"
    ParticleExcel.save(f"./Particle (seed = {seed}).xlsx")
    AveragedExcel.save(f"./Averaged (seed = {seed}).xlsx")


In [ ]:
import cProfile
import pstats

cProfile.run("main()", "./Saved Model/cProFile/cProFile.prof")

# stats = pstats.Stats("./Saved Model/cProFile/cProFile.prof")
# stats.sort_stats("cumtime").print_stats(10)


# plots

In [ ]:
excel_path = "Particle (seed = 30).xlsx"
excel_sheet = "Send Request"
on_off = []
seed = 30
output_name = "o.gif"

Analysis.Make_Animation(excel_path, excel_sheet, ENV_Parameters, Parameters, on_off, seed, output_name)

In [ ]:
excel_name = "30 Averaged MT , lr=1e-5 , alpha4=-1000 , L=45.xlsx"
sheet_name = "Send Request"
output_name = "o.png"


figsize  = (16, 8)
raw_col  = (2 , 6)
labelpad = [0, -6, 2, 2, 2]
ylim     = [[5, 105], [-300000, 0], [10, 3200], [1.05, 13.1], [1.2, 15.99]]  # s
legend   = ['Base', 'Smart Agent', 'Smart Agent Collaboration']
bbox_to_anchor = (0.5, 0., 0.49, 0.38)

figure_feature = [figsize, raw_col, labelpad, ylim, legend, bbox_to_anchor]


importlib.reload(Analysis)
Analysis.Ploting(excel_name, sheet_name, figure_feature, output_name)

# Test

In [103]:
import pandas as pd

excel_name  = "30 Averaged MT , lr=1e-5 , alpha4=-1000 , L=45_.xlsx"
sheets_name = pd.ExcelFile(excel_name).sheet_names

excel_base = pd.read_excel(excel_name, sheets_name[0])
excel_mode = pd.read_excel(excel_name, sheets_name[1])
excel_requ = pd.read_excel(excel_name, sheets_name[2])
excel_ensemble = pd.concat([excel_base, excel_mode.iloc[:, 1:], excel_requ.iloc[:, 1:]], axis=1)
# excel_ensemble.rename(excel_ensemble.columns[1:].to_list().insert(0, 'aa'))

# writer = pd.ExcelWriter("30 Averaged MT , lr=1e-5 , alpha4=-1000 , L=45____.xlsx", engine='openpyxl')
# excel_base.to_excel(writer, sheet_name=sheets_name[0], index=False)
# excel_mode.to_excel(writer, sheet_name=sheets_name[1], index=False)
# excel_requ.to_excel(writer, sheet_name=sheets_name[2], index=False)
# excel_ensemble.to_excel(writer, sheet_name='Ensemble', index=False)
# writer.close()

